In [ ]:
from matplotlib import animation, rc, pyplot as plt
import numpy as np
from numpy import genfromtxt
from scipy.optimize import curve_fit

In [ ]:
wbnlist = genfromtxt('Corrected Potential of 2Electron Ring.csv', delimiter=',',dtype=complex)

In [ ]:
# ------------------------------------------------------------
# Configuration:
# In `__init__`, select either the corrected or uncorrected potential 
# by uncommenting the desired option below
# ------------------------------------------------------------

class QuantumRing:
    def __init__(self,
                 mass = 1, hbar=1,
                 phimin=0, phimax=2*np.pi,ninterval=128):
        
        self.phi = np.linspace(phimin, phimax, ninterval) # discretise the space
        self.dphi = self.phi[1] - self.phi[0]

        self.hbar = hbar
        self.mass = mass
        self.R = 0.5
        self.u = self.R*np.sqrt(2 - 2*np.cos(self.phi-np.pi))
        
        res = ninterval 
        self.dkx = 2 * np.pi / (res * self.dphi)
        self.kx = np.concatenate((np.arange(0, res / 2),
                                 np.arange(-res / 2, 0))) * self.dkx
                
        self.Vhat = np.ones(len(self.phi),dtype=complex)
        #self.Vhat *= 1/self.u   #This is the uncorrected potential.
        self.Vhat *= wbnlist   #This is the corrected potential.
        
        self.Khat = np.ones(len(self.phi),dtype=complex)
        self.Khat *= (2*self.kx**2) /(2 * self.R**2) 
        
        self.wave = self.wave_packet(self.phi)
            
        self.history = {}
    
    def evolve(self, tfinal=6, nt=30000):
        t = np.linspace(0, tfinal, nt)
        dt = t[1] - t[0]

        Vt = np.exp(-1j * (1/2) * self.Vhat * dt / self.hbar)
        Kt = np.exp(-1j * (1/1) * self.Khat * dt / self.hbar)

        t_list=[]
        auto_list=[]        
        
        psi = np.copy(self.wave)

        t_list.append(t[0])
        auto_list.append(np.sum(np.conjugate(self.wave) * psi * self.dphi))
        
        
        for i in range(nt):                     
            
            psi *= Vt 
            
            psi = np.fft.fft(psi)
            psi *= Kt
            psi = np.fft.ifft(psi)
            
            psi *= Vt 
            
            t_list.append(t[i])
            auto_list.append(np.sum(np.conjugate(self.wave) * psi * self.dphi))
            
            
        return t_list, auto_list
                    
    def wave_packet(self, x):
        
        psi = np.ones(len(self.phi),dtype=complex)
        fu = self.u * (1+self.u/(2*self.R))**(1/2) 
        psi *= fu 
        
        norm = np.sum(np.conjugate(psi)*psi*self.dphi)
        psi *= 1/np.sqrt(norm)

        return psi

In [ ]:
qt = QuantumRing()

In [ ]:
t_list, auto_list = qt.evolve()

In [ ]:
#save the data
np.savetxt('2electron ring t_list.csv',t_list, delimiter=',')
np.savetxt('2electron ring autocorrelation.csv',auto_list, delimiter=',')

In [ ]:
#reload the data
t_list = genfromtxt('2electron ring t_list.csv', delimiter=',')
auto_list = genfromtxt('2electron ring autocorrelation.csv', delimiter=',', dtype=np.complex128)

In [ ]:
#plot the real part of the autocorrelation
t = np.array(t_list)         
y_target = np.real(auto_list)     

def model(t, E):
    return np.cos(E * t)           

E0 = 2.0

popt, pcov = curve_fit(model, t, y_target, p0=[E0])
E_fit = popt[0]
E_err = np.sqrt(np.diag(pcov))[0] if pcov is not None else np.nan

print(fr"Fitted E_dynamic = {E_fit:.3f}")

plt.plot(t, y_target, label="Re{A(t)}", lw=1)
plt.plot(t, model(t, E_fit), lw=2,
         label=fr"$\cos({E_fit:.3f}\,t)$")
plt.legend()

plt.ylabel('Real Part of Autocorrelation')
plt.xlabel('Time (a.u.)')
plt.ylim(-1.03, 1.03)
plt.show()

In [ ]:
#plot the absolute autocorrelation
plt.plot(t_list, np.abs(auto_list), linewidth=3)
plt.ylabel('Absolute Autocorrelation')
plt.xlabel('Time (a.u.)')
plt.ylim(0.979, 1.002)